# Chapter 4: Prompting Techniques

---

**Lesson:**

While we have covered the main components of what makes a prompt (positive or negative) strong, there remain a few tweeks we can make to our prompts to ensure we generate the images we desire.

In [ ]:
%%capture
# Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet "pillow>=9.5,<10"

# Python Built-Ins:
import base64
import io
import json
import os
import sys

# External Dependencies:
import boto3
from PIL import Image

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

modelId = "stability.stable-diffusion-xl"

## Keywords and the *weight* they pull

There are 3 techniques central to effective keyword usage in our prompts. Let's dive into each of these one-by-one with accompanying examples.

### Keyword Relevance

Not all words are effective in our prompts. These words serve no purpose in our prompts other than to use up our context window, thereby limiting the number of *useful* keywords we can use. Similarly, some words, while seemingly relevant because they pertain to a specific subject, do not (yet) exist in Stable Diffusion's corpus of knowledge. At the very least, these words are so scarce in the corpus that taught Stable Diffusion that any chance they will produce useful results is of negligible probability. Let's observe with the below example.

**Example 4.1 - Choose your words carefully**

In [ ]:
prompt = "the beautiful dog that is walking to the store in the snow and the sun shining above"

seed = 12345

This is quite a wordy sentence with plenty of words beyond those describing the subject and its surroundings. The question is, are these words *keywords* in the context of Stable Diffusion? We've set the seed to a specific number so we can reduce the probability of variation in our generated images and better compare results.

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt}]
    ),
    "seed": seed,
})

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

os.makedirs("data", exist_ok=True)
eg4_1a = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
eg4_1a.save("data/eg4_1a.png")
eg4_1a

Now, let's try a similar prompt but without the "filler words."

In [ ]:
prompt = "beautiful dog, walking to store in snow, shining sun"

seed = 12345

Notice how we split the prompt into a comma-separated list since Stable Diffusion prefers lists as opposed to sentences as input.

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt}]
    ),
    "seed": seed,
})

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

os.makedirs("data", exist_ok=True)
eg4_1b = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
eg4_1b.save("data/eg4_1b.png")
eg4_1b

### Keyword Weights

Not all keywords are as important as others; this is where *weights* come into play.

We can apply the syntax [{"text": prompt, "weight": factor}] to designate a weight for any given "prompt" variable we have defined, where factor is the weight or importance applied to the word(s) in the variable. A factor less than 1 means Stable Diffusion applies less importance to the word(s) and a factor greater than 1 means the model applies greater importance to the word(s). We have seen this applied previously where our negative prompts were applied a factor of -1.0 to signify they were negative prompts. Let's try this out.

**Example 4.2 - MORE sweetrolls**

In the below, we keep the weights of all keywords at default (i.e., 1).

In [ ]:
prompt = "sweetrolls, scrumptious, glazed, large room, high ceilings"

seed = 12345

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt}]
    ),
    "seed": seed,
})

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

os.makedirs("data", exist_ok=True)
eg4_2a = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
eg4_2a.save("data/eg4_2a.png")
eg4_2a

As you can see, part of the prompt was assigned precendence over the keyword "sweetroll" likely because Stable Diffusion "understands" rooms better than this specific type of food. Now, let's modify the prompt so there is more weight on sweetrolls, thereby increasing the prominance in the image.

In [ ]:
prompt1 = "sweetroll, scrumptious, glazed"
prompt2 = "large room, high ceilings"

seed = 12345

Notice how we must create two separate prompts now since *weight* is applied per prompt.

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt1, "weight": 3.0}]
        + [{"text": prompt2}]
    ),
    "seed": seed,
})

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

os.makedirs("data", exist_ok=True)
eg4_2b = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
eg4_2b.save("data/eg4_2b.png")
eg4_2b

We can also blend keywords via a technique called prompt scheduling. The syntax is [keyword1: keyword2: factor] and what we are doing here is controlling at which *step* in our denoising process keyword1 is switched to keyword2. For example, with factor set to 0.5 and with 30 default steps, the image would begin generation / denoising using keyword1 for steps 1-15, while we would begin "blending" keyword2 into the image from steps 15-30.

**Example 4.3 - If Jeff Bezos and Elon Musk had a child...**

Two tech titans. One image. Let's see what we get!

In [ ]:
prompt = "[Jeff Bezos:Elon Musk:0.5]"

seed = 12345

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt}]
    ),
    "seed": seed,
})

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

os.makedirs("data", exist_ok=True)
eg4_3 = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
eg4_3.save("data/eg4_3.png")
eg4_3

## Exercises:

To bring together all we have learned so far, we will explore some scenarios below and see how we can develop the most optimal images.

**Exercise 4.1 - Company logo**

Yuo have been tasked by your organization to design a logo for a new sustainable line of products. Using proper formatting, the included parameters, detailed descriptions, negative prompting, and weights, generate a logo based on your choice of a sustainable product.

In [ ]:
prompt = "INSERT PROMPT"
negative_prompts = [
    "INSERT NEGATIVE PROMPT",
    "INSERT NEGATIVE PROMPT"
]

cfg_scale = 
seed = 
steps = 
style_preset =   # (e.g. photographic, digital-art, cinematic, ...)
clip_guidance_preset =  # (e.g. FAST_BLUE FAST_GREEN NONE SIMPLE SLOW SLOWER SLOWEST)
sampler =  # (e.g. DDIM, DDPM, K_DPMPP_SDE, K_DPMPP_2M, K_DPMPP_2S_ANCESTRAL, K_DPM_2, K_DPM_2_ANCESTRAL, K_EULER, K_EULER_ANCESTRAL, K_HEUN, K_LMS)
width = 

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt, "weight": 1.0}]
        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
    ),
    "cfg_scale": cfg_scale,
    "seed": seed,
    "steps": steps,
    "style_preset": style_preset,
    "clip_guidance_preset": clip_guidance_preset,
    "sampler": sampler,
    "width": width
})

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

os.makedirs("data", exist_ok=True)
ex4_1 = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
ex4_1.save("data/ex4_1.png")
ex4_1

**Exercise 4.2 - Capturing an image**

Let's revisit this exercise for one final time. See if you can recreate an image of any particular scene in your life, be it your office, park, house, etc. Think about the details required to make this come to life, and incorporate all that we have learned at this point.

In [ ]:
prompt = "INSERT PROMPT"
negative_prompts = [
    "INSERT NEGATIVE PROMPT",
    "INSERT NEGATIVE PROMPT"
]

cfg_scale = 
seed = 
steps = 
style_preset =   # (e.g. photographic, digital-art, cinematic, ...)
clip_guidance_preset =  # (e.g. FAST_BLUE FAST_GREEN NONE SIMPLE SLOW SLOWER SLOWEST)
sampler =  # (e.g. DDIM, DDPM, K_DPMPP_SDE, K_DPMPP_2M, K_DPMPP_2S_ANCESTRAL, K_DPM_2, K_DPM_2_ANCESTRAL, K_EULER, K_EULER_ANCESTRAL, K_HEUN, K_LMS)
width = 

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt, "weight": 1.0}]
        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
    ),
    "cfg_scale": cfg_scale,
    "seed": seed,
    "steps": steps,
    "style_preset": style_preset,
    "clip_guidance_preset": clip_guidance_preset,
    "sampler": sampler,
    "width": width
})

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

os.makedirs("data", exist_ok=True)
ex4_2 = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
ex4_2.save("data/ex4_2.png")
ex4_2